# Create NCSBI 2014-2015 Raw Datasets by County
* This program downloads all original datasets from http://crimereporting.ncsbi.gov/Reports.aspx and saves them as .csv files. These data files are used to create all the flattened and machine learning datasets
    1. This program will dowload specific year data from each web page
    2. It will loop the though 100 counties to get related year data
    3. Get the school list from profile file, then left join with the crime rate got from step 2.
    4. We generate a new csv file and save specific year data for all of the North Carolina counties

In [1]:
# import libaraies
import requests
import urllib.request
import time
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
year = 2015

# root URL for the crime rate in 2017 including 10 years dataset for each county, but we only want to e
rootPath = 'http://crimereporting.ncsbi.gov/public/2017/CrimeTrds/IndexRateCoTrd/report1/'


In [3]:
# Get the header

url = 'http://crimereporting.ncsbi.gov/public/2017/CrimeTrds/IndexRateCoTrd/report1/1.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
# fourth table has the header
table = soup.findAll('table')[4]
headers = []
header = []
header.append('County')

# header is in the first row
for val in table.find_all('tr')[0].find_all('td'):
    #print(val.text)
    header.append(val.text)
headers.append(header)

In [4]:
def getCrimeDataByYear(url,yearToExtract):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.findAll('table')[3]
    county = title.findAll('td')[0].text + ' County'
    
    table = soup.findAll('table')[4]
   
    for row in table.find_all('tr')[1:]:

        data =[]
        data.append(county)
        firstCol = True
        for val in row.find_all('td'):
            if(firstCol==True):
                yearVal = int(val.text)
            fieldVal = val.text
            if(fieldVal == '\xa0'):
                fieldVal = ''
            data.append(fieldVal)
            firstCol=False
        #print(yearVal,year)

        if(yearVal==yearToExtract):
            #print('true')
            crime.loc[len(crime)] = data
    
    #print(headers)
    #print(rows)

In [5]:
# Loop for all of the counties to get the data for specific year
crime =pd.DataFrame(columns = list(headers)[0])
for i in range(1,101):
    url = rootPath+str(i)+'.htm'
    getCrimeDataByYear(url,year)

crime.columns = ['County','Year','Crime_Rate_Index_Rate'
                ,'Crime_Rate_Violent_Rate','Crime_Rate_Property_Rate','Crime_Rate_Murder_Rate'
                ,'Crime_Rate_Rape_Rate','Crime_Rate_Robbery_Rate'
                ,'Crime_Rate_Assault_Rate','Crime_Rate_Burglary_Rate'
                ,'Crime_Rate_Larceny_Rate','Crime_Rate_MVT_Rate','Crime_Rate_Arson_Rate']
    
# Import the profile 
inputFile = '../../Raw Datasets/profile.csv'
profile = pd.read_csv(inputFile, low_memory=False, dtype={'unit_code': object})
# get the lea 
lea = profile[['unit_code','School_Name','Lea_Name']].drop_duplicates()
lea['County'] = lea['Lea_Name'].str.replace(r' Schools', '')
lea = lea.sort_values(by=['County'])

# join the profile with crime dataset
crime_combined = pd.merge(lea , crime, on = 'County', how = 'left')


In [6]:
# save the crime data frome to csv file
crime_combined.to_csv(r'../CrimeData2015.csv',index = False)